In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [4]:
import xarray as xar

D:\01Setup\ANACONDA\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [5]:
file_path='./data/sanjioil.nc'

In [6]:
ds=nc.Dataset(file_path)

In [7]:
ds_xr=xar.open_dataset(file_path)

In [8]:
ds_xr.dims

Frozen(SortedKeysDict({'trajectory': 2958, 'time': 73}))

In [9]:
ds_xr.data_vars

Data variables:
    age_seconds                               (trajectory, time) float32 ...
    interfacial_area                          (trajectory, time) float32 ...
    mass_evaporated                           (trajectory, time) float32 ...
    water_fraction                            (trajectory, time) float32 ...
    y_wind                                    (trajectory, time) float32 ...
    sea_ice_area_fraction                     (trajectory, time) float32 ...
    density                                   (trajectory, time) float32 ...
    oil_film_thickness                        (trajectory, time) float32 ...
    x_sea_water_velocity                      (trajectory, time) float32 ...
    bulltime                                  (trajectory, time) float32 ...
    x_wind                                    (trajectory, time) float32 ...
    age_emulsion_seconds                      (trajectory, time) float32 ...
    sea_surface_wave_stokes_drift_y_velocity  (trajectory, t

In [10]:
ds_xr.coords

Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
  * time        (time) datetime64[ns] 2018-01-14T22:20:00 ... 2018-01-17T22:20:00
    lon         (trajectory, time) float32 ...
    lat         (trajectory, time) float32 ...

## 获取`几个指定维度的`的`DataArray`  
如下可知：
    `mass_oil`有2958个值，该物理量有两个维度，分别为`trajectory`,`time`

In [11]:
# 读取一个 wind 
ds_xr.isel(time=1)['x_wind']

<xarray.DataArray 'x_wind' (trajectory: 2958)>
array([-6.335222e+00, -7.076474e+00, -5.017034e+00, ...,  9.969210e+36,
        9.969210e+36,  9.969210e+36], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-14T23:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...

### 1- 尝试将xarray转为series

## 2 剔除掉nan的数据，只获取有效数据

#### 查看不同时刻的
并通过`where`剔除掉大于`max`以及`nan`的值  
使用xarray的dropna方法，注意与pandas中的方法有所区别

提示错误：
`ValueError: any must be a single dataset dimension`  
查看文档后发现`xarray.Dataset.dropna`尚不支持同时沿多个维度放置  

[参考](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.dropna.html)  

考虑将xarray转换为dateframe，然后再清洗？  
[上网参考](https://stackoverflow.com/questions/52553925/python-xarray-remove-coordinates-with-all-missing-variables)
比较好的解决办法是：
1. 将`DataSet`->`DataFrame`
2. 在`DataFrame`中通过`pandas` `dropna` 即可

In [38]:
xr_temp_x=ds_xr.isel(time=30)['x_wind']
xr_temp_x

<xarray.DataArray 'x_wind' (trajectory: 2958)>
array([ 9.969210e+36, -2.856885e+00,  9.969210e+36, ...,  9.969210e+36,
        9.969210e+36,  9.969210e+36], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...

In [39]:
xr_temp_y=ds_xr.isel(time=30)['y_wind']
xr_temp_y

<xarray.DataArray 'y_wind' (trajectory: 2958)>
array([9.969210e+36, 1.047276e+01, 9.969210e+36, ..., 9.969210e+36,
       9.969210e+36, 9.969210e+36], dtype=float32)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...

* 注意若使用`.concat`合并的将会是追加，append的操作

In [40]:
xar.concat([xr_temp,xr_temp_y],dim='trajectory')

<xarray.DataArray 'x_wind' (trajectory: 5916)>
array([ 9.9692100e+36, -2.8568854e+00,  9.9692100e+36, ...,
        9.9692100e+36,  9.9692100e+36,  9.9692100e+36], dtype=float32)
Coordinates:
    time        datetime64[ns] 2018-01-16T04:20:00
  * trajectory  (trajectory) int64 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    lon         (trajectory) float32 9.96921e+36 125.85207 ... 9.96921e+36
    lat         (trajectory) float32 9.96921e+36 28.504345 ... 9.96921e+36

以下出现错误：
 MergeError: conflicting values for variable 'time' on objects to be combined. You can skip this check by specifying compat='override'.  
 出现此错误经检查为 `time` 不是同一个time导致的  
 #### 最终使用merage的方式将两个维度进行合并

In [44]:
xr_merge=xar.merge([xr_temp_x,xr_temp_y])
xr_merge

<xarray.Dataset>
Dimensions:     (trajectory: 2958)
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 9.96921e+36 125.85207 ... 9.96921e+36
    lat         (trajectory) float32 9.96921e+36 28.504345 ... 9.96921e+36
Data variables:
    x_wind      (trajectory) float32 9.96921e+36 -2.8568854 ... 9.96921e+36
    y_wind      (trajectory) float32 9.96921e+36 10.472756 ... 9.96921e+36

In [47]:
# 剔除大于阈值的，并且为nan的
xr_merge.where(xr_merge<2000).to_dataframe().dropna(how='any')

,x_wind,y_wind,time,lon,lat
trajectory,,,,,
2,-2.856885,10.472756,2018-01-16 04:20:00,125.852074,28.504345
4,-3.510324,8.823525,2018-01-16 04:20:00,125.837547,28.535669
5,-3.477507,8.365860,2018-01-16 04:20:00,125.746597,28.534342
9,-3.867782,9.866551,2018-01-16 04:20:00,125.779015,28.548067
10,-4.172445,8.833652,2018-01-16 04:20:00,125.941368,28.543129
...,...,...,...,...,...
1250,-2.741773,11.013288,2018-01-16 04:20:00,125.921379,28.369308
1251,-4.936096,8.763128,2018-01-16 04:20:00,125.921432,28.369282
1252,-1.048339,10.038569,2018-01-16 04:20:00,125.921486,28.369307
